# Grasple Test to Microsoft Excel

Below you can upload a `.csv`-file exported from a Grasple test. After review of the generated ouput you can converted the output to a Microsoft Excel file available for download.

No data is stored on the server, as all Python code is run within your local browser. All data will be cleared after you close the browser tab.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
import base64
import numpy as np
import micropip
import threading
from IPython import get_ipython

# Widgets
upload = widgets.FileUpload(accept='.csv', multiple=False)
output = widgets.Output()
download_button = widgets.Button(description="Generate Excel file", disabled=True)
output_link = widgets.Output()

# Install xlsxwriter using micropip
await micropip.install("xlsxwriter")

# Global variable to store the timer
reset_timer = None

def create_download_link(df):
    """Create a downloadable link for the DataFrame as an Excel file."""
    with io.BytesIO() as buffer:
        with pd.ExcelWriter(buffer, engine="xlsxwriter") as writer:
            df.to_excel(writer, index=True, sheet_name="Sheet1")
        buffer.seek(0)
        b64 = base64.b64encode(buffer.read()).decode()
        href = f'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}'
        return href

def reset_kernel():
    """Reset the Thebe kernel and clear all outputs."""
    print("⏳ Resetting kernel and clearing memory after 5 minutes...")
    get_ipython().kernel.do_shutdown(restart=True)  # Restart the kernel

def start_reset_timer():
    """Start a timer to reset the kernel after 5 minutes."""
    global reset_timer
    if reset_timer:
        reset_timer.cancel()  # Cancel any existing timer
    reset_timer = threading.Timer(300, reset_kernel)  # 300 seconds = 5 minutes
    reset_timer.start()

def on_download_click(b):
    """Handle the download button click."""
    output_link.clear_output()
    with output_link:
        try:
            # Create the download link
            download_link = create_download_link(pvt)
            # Display the download link
            display(widgets.HTML(
                f'<a download="output.xlsx" href="{download_link}" target="_blank">Download Excel</a>'
            ))
        except Exception as e:
            print("❌ Error creating download link:")
            print(e)

def on_upload_change(change):
    output.clear_output()
    global pvt  # Declare pvt as global to use it in the download button
    if upload.value:
        with output:
            try:
                # Retrieve the first file from the tuple
                uploaded_file = upload.value[0] if isinstance(upload.value, tuple) else list(upload.value.values())[0]
                
                # In some environments, upload.value[0] is a dictionary.
                if isinstance(uploaded_file, dict):
                    content = uploaded_file['content']
                else:
                    # Sometimes it's a tuple (name, file dict)
                    content = uploaded_file[1]['content']
                
                df = pd.read_csv(io.BytesIO(content))
                df = df.drop(df[df['started_test'] == 0].index)
                df['main_exercise_id'] = df['main_exercise_id'].astype(int)
                print("✅ CSV successfully loaded.")
                
                # Convert to pivot table
                question_pool_names_temp = df['question_pool_name'].unique()
                question_pool_names = [
                    name for name in question_pool_names_temp if isinstance(name, str)
                ]
                
                main_exercise_ids_temp = df['main_exercise_id'].unique()
                main_exercise_ids = [
                    number for number in main_exercise_ids_temp if isinstance(number, str)
                ]
                
                if question_pool_names:
                    pvt = pd.pivot_table(
                        df, 
                        values="scored_points", 
                        index=["student_id", "user_name"], 
                        aggfunc="sum", 
                        fill_value=0, 
                        columns="question_pool_name"
                    )
                    times = pd.pivot_table(
                        df, 
                        values="exercise_received_at", 
                        index="student_id", 
                        aggfunc="min", 
                        fill_value="missing", 
                        columns="question_pool_name"
                    )
                    for name in question_pool_names:
                        times = times.drop(times[times[name] == "missing"].index)
                    times = times.iloc[:1].T
                    times.rename(columns={times.columns[0]: 'times'}, inplace=True)
                    times.sort_values(by='times', inplace=True)
                    pvt = pvt[times.index]
                else:
                    pvt = pd.pivot_table(
                        df, 
                        values="scored_points", 
                        index=["student_id", "user_name"], 
                        aggfunc="sum", 
                        fill_value=0, 
                        columns="main_exercise_id"
                    )
                    times = pd.pivot_table(
                        df, 
                        values="exercise_received_at", 
                        index="student_id", 
                        aggfunc="min", 
                        fill_value="missing", 
                        columns="main_exercise_id"
                    )
                    for name in main_exercise_ids:
                        times = times.drop(times[times[name] == "missing"].index)
                    times = times.iloc[:1].T
                    times.rename(columns={times.columns[0]: 'times'}, inplace=True)
                    times.sort_values(by='times', inplace=True)
                    pvt = pvt[times.index]
                
                display(pvt)
                download_button.disabled = False  # Enable the download button
                start_reset_timer()  # Start the timer to reset the kernel
            except Exception as e:
                print("❌ Error loading CSV:")
                print(e)

upload.observe(on_upload_change, names='value')
download_button.on_click(on_download_click)

# Display the widgets
display(upload, output, download_button, output_link)